In [2]:
# import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# define the paths to the train and test folders on your desktop
train_path = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/train'
test_path = 'C:/Users/chand/Desktop/New folder/DeepLearning/Dataset/test'

# define image and batch size
img_height, img_width = 28, 28
batch_size = 64

# create ImageDataGenerator objects for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

# create flow_from_directory generators for train and test datasets
train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(img_height, img_width),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

# define the model architecture
model = keras.Sequential(
    [
        keras.Input(shape=(img_height, img_width, 1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(train_generator.num_classes, activation="softmax"),
    ]
)

# compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# train the model
# MODEL.summary

history = model.fit(train_generator, epochs=10, validation_data=test_generator)

# evaluate the model on test set
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print("Test accuracy:", test_acc)


Found 17000 images belonging to 10 classes.
Found 2999 images belonging to 10 classes.
Epoch 1/10
266/266 [==============================] - 17s 61ms/step - loss: 1.1045 - accuracy: 0.6404 - val_loss: 0.2834 - val_accuracy: 0.9183
Epoch 2/10
266/266 [==============================] - 16s 61ms/step - loss: 0.5400 - accuracy: 0.8311 - val_loss: 0.1665 - val_accuracy: 0.9463
Epoch 3/10
266/266 [==============================] - 16s 59ms/step - loss: 0.4022 - accuracy: 0.8720 - val_loss: 0.1609 - val_accuracy: 0.9480
Epoch 4/10
266/266 [==============================] - 16s 60ms/step - loss: 0.3276 - accuracy: 0.8976 - val_loss: 0.1185 - val_accuracy: 0.9670
Epoch 5/10
266/266 [==============================] - 16s 59ms/step - loss: 0.2830 - accuracy: 0.9112 - val_loss: 0.0824 - val_accuracy: 0.9747
Epoch 6/10
266/266 [==============================] - 16s 60ms/step - loss: 0.2595 - accuracy: 0.9185 - val_loss: 0.0776 - val_accuracy: 0.9760
Epoch 7/10
266/266 [=============================

In [3]:
# define a list of optimizers to try
optimizers = [tf.keras.optimizers.SGD(),tf.keras.optimizers.Adam(),tf.keras.optimizers.Adadelta()]

# loop through each optimizer and train the model
for optimizer in optimizers:
    
    # reset the model weights for each optimizer
    model = keras.Sequential(
        [
            keras.Input(shape=(img_height, img_width, 1)),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation="relu"),
            layers.Dense(train_generator.num_classes, activation="softmax"),
        ]
    )
    
    # compile the model with the current optimizer
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    
    # train the model
    history = model.fit(train_generator, epochs=10, validation_data=test_generator, verbose=0)
    
    # evaluate the model on test set
    test_loss, test_acc = model.evaluate(test_generator, verbose=0)
    test_predictions = model.predict(test_generator)
    test_pred_labels = np.argmax(test_predictions, axis=1)
    test_true_labels = test_generator.classes
    
    # calculate precision, recall, and F-score
    precision = tf.keras.metrics.Precision()(test_true_labels, test_pred_labels)
    recall = tf.keras.metrics.Recall()(test_true_labels, test_pred_labels)
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    # print the evaluation metrics
    print("Optimizer:", optimizer.__class__.__name__)
    print("Test loss:", test_loss)
    print("Test accuracy:", test_acc)
    print("Precision:", precision.numpy())
    print("Recall:", recall.numpy())
    print("F-score:", f1_score.numpy())
    print("\n")


47/47 [==============================] - 3s 29ms/step
Optimizer: SGD
Test loss: 0.3211318254470825
Test accuracy: 0.9033011198043823
Precision: 0.99776953
Recall: 0.99407405
F-score: 0.9959184


47/47 [==============================] - 2s 49ms/step
Optimizer: Adam
Test loss: 0.04432033747434616
Test accuracy: 0.9866622090339661
Precision: 0.9996298
Recall: 1.0
F-score: 0.99981487


47/47 [==============================] - 2s 49ms/step
Optimizer: Adadelta
Test loss: 2.0464608669281006
Test accuracy: 0.489829957485199
Precision: 0.9178316
Recall: 0.99703705
F-score: 0.9557963


